#### Just libraries 

In [2]:
from bs4 import BeautifulSoup
import re

## Reading from a single file
This code will work only on one arbitrary file, it's isefull just for try if the code works or not.

In [3]:
with open("Movies\\movie0.html", 'rb') as html:
    soup = BeautifulSoup(html)

In [11]:
title = soup.title.string.strip(" - Wikipedia") #taking the title, without the " - Wikipedia" last part.

body = soup.body #put the soup body in a variable.
paragraphNumber = 1 #a counter for know whitch paragraf are we working on 
for paragraph in body.find_all('p'): #iterating on the paragrafs 
    if(paragraphNumber == 1):#putting the first paragraf in the intro variable
        intro = paragraph.text.strip() #using strip to delete space characters
        paragraphNumber += 1
    elif(paragraphNumber == 2):#putting the second paragraf in the plot variable
        plot = paragraph.text.strip() #using strip to delete space characters
        break #we don't need the other paragrafs


#just printing for see some results:
print ("Title: ", title, "\n")
print ("Intro: ", intro, "\n")
print ("Plot: ", plot, "\n")

Title:  Love by the Light of the Moon 

Intro:  Love by the Light of the Moon is a 1901 film by Edwin S. Porter, produced by the Edison Manufacturing Company.[1] It mixes animation and live action and predates the man in the moon theme of the 1902 French science fiction film A Trip to the Moon by Georges Méliès. 

Plot:  The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better. 



## writing in the file 
This code write a list of aguments pn the tsv file.

In [12]:
import csv
with open('outputtest.tsv', 'wt') as out_file: #creating one file for each different movie 
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow([title, intro, plot]) #input to the tsv file.

## Reading from the files and witing the the others
Here the code that should be able to iterate through all the files.

In [ ]:

fileNumber = 0
while os.path.exists("Movies\\movie" + fileNumber + ".html"): #Iterating for each file 
    
    #Parsing the html file:
    with open("Movies\\movie" + fileNumber + ".html", 'rb') as html:
        soup = BeautifulSoup(html)
        
    title = soup.title.string.strip(" - Wikipedia") #taking the title, without the " - Wikipedia" last part.

    body = soup.body #put the soup body in a variable.
    paragraphNumber = 1 #a counter for know whitch paragraf are we working on 
    for paragraph in body.find_all('p'): #iterating on the paragrafs 
        if(paragraphNumber == 1):#putting the first paragraf in the intro variable
            intro = paragraph.text.strip() #using strip to delete space characters
            paragraphNumber += 1
        elif(paragraphNumber == 2):#putting the second paragraf in the plot variable
            plot = paragraph.text.strip() #using strip to delete space characters
            break #we don't need the other paragrafs
    
    """we still must cover the case when the wikipedia page has less than two paragraphs, 
    and so ether plot or both intro and plot are not assigned."""

    
    #writing the results in the tsv file 
    import csv
    with open('article_' + fileNumber + '.tsv', 'wt') as out_file: #creating one file for each different movie 
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow([title, intro, plot]) #input to the tsv file.
    
    """code for save on the file works."""
    
    fileNumber += 1 #let's move to the next file!